# Klasyfikacja kwiatów


Ten zestaw danych zawiera 9 różnych rodzajów kwiatów. Twoim zadaniem jest zbudowanie modelu uczenia maszynowego lub głębokiego uczenia, który będzie w stanie klasyfikować te obrazy jako kwiaty. Oto klasy, które są dostępne:

    Tulipan
    Słonecznik
    Róża
    Orchidea
    Lotos
    Lilia
    Lawenda
    Mniszek lekarski
    Stokrotka

Twoim zadaniem jest stworzenie modelu sieci neuronowej, który będzie klasyfikować obrazy w kategorie kwiatów np. tulipan itp. W tym celu wykorzystaj model ResNet50, który został wstępnie przeszkolony na szerokim zbiorze danych.

Kroki, które musisz podjąć:

1. **Załadowanie danych**: Dane treningowe i walidacyjne zostały już przygotowane w formie generatorów za pomocą biblioteki `ImageDataGenerator`. Generator treningowy zawiera obrazy treningowe, a generator walidacyjny zawiera obrazy walidacyjne. Obrazy są przeskalowane do rozmiaru `(224, 224)` i podzielone na dwie klasy: koty i psy.

2. **Model ResNet50**: W kodzie załaduj model ResNet50, który jest wstępnie przeszkolony na zestawie danych ImageNet. Model ten ma zamrożone wagi wszystkich warstw.

3. **Warstwy klasyfikacji**: Do modelu ResNet50 dodano warstwy klasyfikacji. Jest to warstwa `GlobalAveragePooling2D` i warstwa `Dense`, która wykorzystuje funkcję aktywacji softmax. Model ten jest przystosowany do zadania klasyfikacji.

4. **Kompilacja modelu**: Model jest skompilowany z optymalizatorem Adam, funkcją straty "categorical_crossentropy" (odpowiednia do klasyfikacji wieloklasowej), oraz metryką "accuracy".

5. **Trenowanie modelu**: Model jest trenowany na danych treningowych przez jedną epokę. Wynik trenowania jest wyświetlany na konsoli.

Twoim zadaniem jest uzupełnienie tego kodu o brakujące elementy, takie jak liczba epok trenowania, odblokowanie warstw modelu ResNet50 oraz dostosowanie modelu do odpowiedniego rodzaju klasyfikacji (wieloklasowej). Następnie możesz dostosować parametry trenowania, takie jak liczba epok czy wielkość wsadu (batch size), aby osiągnąć jak najlepsze wyniki w zadaniu klasyfikacji.

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Ustawienie ścieżki do katalogu z danymi

# Definicja generatora obrazów
datagen = ImageDataGenerator(
    rescale=1.0/255, # Przeskalowanie wartości pikseli do zakresu [0, 1]
    validation_split=0.2 # Procent danych przeznaczonych na zbiór walidacyjny
)

# Wczytanie danych treningowych
train_generator = datagen.flow_from_directory(
    '/kaggle/input/national-flowers/flowerdataset/train',
    target_size=(224, 224), # Rozmiar docelowy obrazów
    batch_size=32, # Rozmiar wsadu
    class_mode='categorical', # Rodzaj klasyfikacji (binary lub categorical)
    subset='training' # Ustawienie dla danych treningowych
)


2024-02-27 17:34:41.879481: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-27 17:34:41.879610: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-27 17:34:42.056549: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found 2880 images belonging to 9 classes.


In [3]:
# Wczytanie danych treningowych
validation_generator = datagen.flow_from_directory(
    '/kaggle/input/national-flowers/flowerdataset/test',
    target_size=(224, 224), # Rozmiar docelowy obrazów
    batch_size=32, # Rozmiar wsadu
    class_mode='categorical', # Rodzaj klasyfikacji (binary lub categorical)
    subset='training' # Ustawienie dla danych treningowych
)


Found 706 images belonging to 9 classes.


In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from keras.layers import Input, Dense, GlobalAveragePooling2D, AveragePooling2D, Flatten


In [5]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

# Adding new classification layers
model = Sequential([   base_model,   GlobalAveragePooling2D(),   Dense(9, activation='softmax')])
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_generator, epochs=10, verbose=1, validation_data=validation_generator)

94765736/94765736 [==============================] - 0s 0us/step
Epoch 1/10
 1/90 [..............................] - ETA: 11:45 - loss: 2.3100 - accuracy: 0.2188

I0000 00:00:1709055314.000888     116 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


90/90 [==============================] - 57s 553ms/step - loss: 2.1709 - accuracy: 0.1628 - val_loss: 2.1364 - val_accuracy: 0.1615
Epoch 2/10
90/90 [==============================] - 17s 182ms/step - loss: 2.0975 - accuracy: 0.2125 - val_loss: 2.0919 - val_accuracy: 0.1955
Epoch 3/10
90/90 [==============================] - 19s 211ms/step - loss: 2.0604 - accuracy: 0.2316 - val_loss: 2.0459 - val_accuracy: 0.2337
Epoch 4/10
90/90 [==============================] - 16s 182ms/step - loss: 2.0330 - accuracy: 0.2451 - val_loss: 2.0265 - val_accuracy: 0.1997
Epoch 5/10
90/90 [==============================] - 17s 184ms/step - loss: 2.0071 - accuracy: 0.2694 - val_loss: 2.0279 - val_accuracy: 0.2181
Epoch 6/10
90/90 [==============================] - 18s 203ms/step - loss: 1.9915 - accuracy: 0.2795 - val_loss: 1.9863 - val_accuracy: 0.2734
Epoch 7/10
90/90 [==============================] - 17s 184ms/step - loss: 1.9718 - accuracy: 0.2979 - val_loss: 2.0024 - val_accuracy: 0.2578
Epoch 8/10

### 

# Detekcja fake newsów

Ten zbiór danych posiada 6 atrybutów, spośród których Nagłówek_Wiadomości jest dla nas najważniejszy, aby klasyfikować wiadomości jako FAŁSZYWE lub PRAWDZIWE. Jak zauważysz w atrybucie Etykieta, określono w nim 6 klas. Dlatego to od Ciebie zależy, czy chcesz użyć mojego zbioru danych do klasyfikacji wieloklasowej, czy przekształcić te etykiety klas na FAŁSZ lub PRAWDA, a następnie przeprowadzić klasyfikację binarną. Chociaż dla Twojej wygody, napiszę notatnik na temat tego, jak przekształcić ten zbiór danych z wieloklasowego na dwuklasowy. Aby radzić sobie z danymi tekstowymi, musisz mieć dobre praktyczne doświadczenie w zakresie NLP (Przetwarzanie Języka Naturalnego) i koncepcji Data-Miningu (Eksploracji Danych).

    Nagłówek_Wiadomości - zawiera fragment informacji, który ma być analizowany.
    Link_Do_Wiadomości - zawiera adres URL nagłówków wiadomości określonych w pierwszej kolumnie.
    Źródło - ta kolumna zawiera nazwiska autorów, którzy opublikowali informacje na Facebooku, Instagramie, Twitterze lub innej platformie mediów społecznościowych.
    Data_Opublikowania - Ta kolumna zawiera datę, kiedy informacje zostały opublikowane przez autorów na różnych platformach mediów społecznościowych.
    Data - Ta kolumna zawiera datę, kiedy ta informacja została przeanalizowana przez zespół Politifact do sprawdzania faktów w celu oznaczenia jako FAŁSZYWE lub PRAWDZIWE.
    Etykieta - Ta kolumna zawiera 5 etykiet klas: Prawda, Większość-Prawdy, Pół-Prawdy, Ledwo-Prawdy, Fałsz, Spodnie w Ogniu.


### Zadanie prostsze: Analiza fake newsów na podstawie tytułów z PositionalEncoding


Twoim zadaniem jest stworzenie modelu sieci neuronowej, który będzie klasyfikować tweety na podstawie tytułu. W danych mamy dostarczone tytuły artykułów oraz etykiety, zdecyduj co z nimi zrobić :). 
Kroki, które musisz podjąć:

1. Przygotowanie danych: Dane zostały wczytane z pliku CSV. Teksty znajdują się w kolumnach title, a etykiety w kolumnie "class". Teksty oraz etykiety są przygotowane do dalszej obróbki.

2. Tokenizacja tekstu: Teksty powinny być tokenizowane za pomocą Tokenizer, a następnie przekształcane na sekwencje liczb. Jest to niezbędne, aby można było używać ich jako danych wejściowych do modelu sieci neuronowej.

3. Zbuduj model sieci neuronowej: Model sieci rekurencyjnej zdefiniuj w kodzie i przykładowo niech składa się z następujących warstw:

* Warstwa wejściowa: Przyjmuje sekwencje tokenów o długości określonej przez max_sequence_length.
* Warstwa Embedding: Mapuje tokeny na wektory o wymiarze embed_dim (100 w tym przypadku), co pozwala na reprezentację słów jako gęstych wektorów cech.
* Warstwa MultiHeadAttention: Implementuje mechanizm uwagi, który pozwala modelowi skupić się na różnych częściach sekwencji wejściowej, co jest szczególnie przydatne w przetwarzaniu języka naturalnego.
* GlobalAveragePooling1D: Agreguje informacje z różnych części sekwencji, redukując wymiarowość danych i przygotowując je do procesu klasyfikacji.
* Warstwa Dense (wyjściowa): Z funkcją aktywacji softmax, generuje dystrybucję prawdopodobieństwa dla dwóch możliwych klas sentymentu, umożliwiając klasyfikację.


4. Kompilacja modelu: Model jest kompilowany z optymalizatorem Adam, funkcją straty "categorical_crossentropy" (odpowiednia do klasyfikacji wieloklasowej) oraz metryką "accuracy".

5. Trenowanie modelu: Model jest trenowany na danych tweetów przez 5 epok, z wsadem o rozmiarze 32. Wynik trenowania jest oceniany na zbiorze walidacyjnym.

Twoim zadaniem jest uzupełnienie kodu i dostosowanie parametrów modelu oraz trenowania, aby uzyskać jak najlepsze wyniki w zadaniu analizy fake newsów na podstawie tytułów. Możesz eksperymentować z parametrami, takimi jak liczba epok trenowania, rozmiar wsadu (batch size).


In [7]:
import pandas as pd 
file_path = '/kaggle/input/fakereal-news/New Task.csv'
try:
    df = pd.read_csv(file_path, encoding='utf-8')
except UnicodeDecodeError:
    try:
        df = pd.read_csv(file_path, encoding='ISO-8859-1')  # Próba z kodowaniem ISO-8859-1
    except UnicodeDecodeError:
        df = pd.read_csv(file_path, encoding='Windows-1252')  # Próba z kodowaniem Windows-1252

df.head()


,News_Headline,Link_Of_News,Source,Stated_On,Date,Label
0,Says Osama bin Laden endorsed Joe Biden,https://www.politifact.com/factchecks/2020/jun...,Donald Trump Jr.,"June 18, 2020","June 19, 2020",FALSE
1,CNN aired a video of a toddler running away fr...,https://www.politifact.com/factchecks/2020/jun...,Donald Trump,"June 18, 2020","June 19, 2020",pants-fire
2,Says Tim Tebow kneeled in protest of abortion...,https://www.politifact.com/factchecks/2020/jun...,Facebook posts,"June 12, 2020","June 19, 2020",FALSE
3,Even so-called moderate Democrats like Joe Bi...,https://www.politifact.com/factchecks/2020/jun...,Paul Junge,"June 10, 2020","June 19, 2020",barely-true
4,"""Our health department, our city and our count...",https://www.politifact.com/factchecks/2020/jun...,Jeanette Kowalik,"June 14, 2020","June 18, 2020",TRUE


In [8]:
df['Label'].unique()

array(['FALSE', 'pants-fire', 'barely-true', 'TRUE', 'mostly-true',
       'half-true', 'full-flop', 'half-flip', 'no-flip'], dtype=object)

In [9]:
X=df['News_Headline'].to_numpy()
labels = pd.get_dummies(df['Label']).values
y=labels

In [10]:
y

array([[ True, False, False, ..., False, False, False],
       [False, False, False, ..., False, False,  True],
       [ True, False, False, ..., False, False, False],
       ...,
       [False, False, False, ...,  True, False, False],
       [False, False,  True, ..., False, False, False],
       [False,  True, False, ..., False, False, False]])

In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# Tokenizacja tekstu
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
word_index = tokenizer.word_index
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(sequences, y, test_size=0.3, random_state=42)

max_sequence_length = max(len(seq) for seq in X)
X_train = pad_sequences(X_train, maxlen=max_sequence_length)
X_val = pad_sequences(X_val, maxlen=max_sequence_length)

In [14]:
from tensorflow.keras.layers import Layer, Embedding, Dense, Dropout, LayerNormalization
from tensorflow.keras.models import Model
from tensorflow.keras import Input

class PositionalEncoding(Layer):
   def __init__(self, position, d_model):
       super(PositionalEncoding, self).__init__()
       self.pos_encoding = self.positional_encoding(position, d_model)

   def get_angles(self, position, i, d_model):
       angles = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
       return position * angles

   def positional_encoding(self, position, d_model):
       angle_rads = self.get_angles(np.arange(position)[:, np.newaxis],
                                    np.arange(d_model)[np.newaxis, :],
                                    d_model)
       # Aplikuj sinus do indeksów parzystych w macierzy; 2i
       angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
       # Aplikuj cosinus do indeksów nieparzystych w macierzy; 2i+1
       angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
       pos_encoding = angle_rads[np.newaxis, ...]
       return tf.cast(pos_encoding, dtype=tf.float32)

   def call(self, inputs):
       return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]


In [15]:
from tensorflow.keras.layers import Layer, Embedding, Dense, Dropout, LayerNormalization
from tensorflow.keras.models import Model
from tensorflow.keras import Input

from keras import layers
class TokenAndPositionEmbedding(layers.Layer):
   def __init__(self, maxlen, vocab_size, embed_dim):
       super().__init__()
       self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
       self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

   def call(self, x):
       # Obliczanie długości sekwencji
       maxlen = tf.shape(x)[-1]
       # Generowanie pozycji dla każdego tokenu w sekwencji
       positions = tf.range(start=0, limit=maxlen, delta=1)
       # Uzyskanie pozycyjnego embedding
       positions = self.pos_emb(positions)
       # Uzyskanie tlenowego embedding
       x = self.token_emb(x)
       # Dodanie pozycyjnego embedding do tlenowego embedding
       return x + positions



In [20]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Embedding, MultiHeadAttention, GlobalAveragePooling1D, Dense
import numpy as np
embed_dim = 100  # Wymiar wektora embeddingowego
num_heads = 5  # Liczba "głowic" w warstwie Multi-Head Attention
vocab_size = 10000  # Rozmiar słownika

inputs = Input(shape=(max_sequence_length,))
#embedding_layer = Embedding(input_dim=vocab_size, output_dim=embed_dim)(inputs)
positional_layer=TokenAndPositionEmbedding(max_sequence_length,vocab_size, embed_dim)(inputs)
#positional_layer=PositionalEncoding(vocab_size, embed_dim)(inputs),
attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(positional_layer, positional_layer)
global_average = GlobalAveragePooling1D()(attention_output)
global_average= Dropout(0.5) (global_average)
#dense_layer = Dense(9, activation='relu')(global_average)
outputs = Dense(9, activation='softmax')(global_average)

model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)


Epoch 1/20
175/175 [==============================] - 24s 77ms/step - loss: 1.8745 - accuracy: 0.1967 - val_loss: 1.8221 - val_accuracy: 0.2280
Epoch 2/20
175/175 [==============================] - 9s 50ms/step - loss: 1.8407 - accuracy: 0.2006 - val_loss: 1.8133 - val_accuracy: 0.2280
Epoch 3/20
175/175 [==============================] - 8s 47ms/step - loss: 1.8289 - accuracy: 0.2057 - val_loss: 1.8120 - val_accuracy: 0.2280
Epoch 4/20
175/175 [==============================] - 8s 47ms/step - loss: 1.8193 - accuracy: 0.2130 - val_loss: 1.8025 - val_accuracy: 0.2301
Epoch 5/20
175/175 [==============================] - 8s 45ms/step - loss: 1.7472 - accuracy: 0.2578 - val_loss: 1.8114 - val_accuracy: 0.2215
Epoch 6/20
175/175 [==============================] - 8s 44ms/step - loss: 1.6228 - accuracy: 0.3072 - val_loss: 1.7541 - val_accuracy: 0.2495
Epoch 7/20
175/175 [==============================] - 7s 42ms/step - loss: 1.4872 - accuracy: 0.3660 - val_loss: 1.9051 - val_accuracy: 0.245

Model sie przeucza bardzo szybko, niezależnie od parametrów

#  Binary Classification Amazon Reviews

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from keras.layers import Input, Dense, GlobalAveragePooling2D, AveragePooling2D, Flatten

dataset_train = pd.read_csv('../input/amazon-reviews-for-sa-binary-negative-positive-csv/amazon_review_sa_binary_csv/train.csv')
dataset_test = pd.read_csv('../input/amazon-reviews-for-sa-binary-negative-positive-csv/amazon_review_sa_binary_csv/test.csv')


In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
X=dataset_train['review_title'].values.astype(str)
y=dataset_train['class_index'].values



In [7]:
X.shape

(3600000,)

In [8]:
def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    [s.extract() for s in soup(['iframe', 'script'])]
    stripped_text = soup.get_text()
    stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
    return stripped_text

def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text



def pre_process_corpus(docs):
    norm_docs = []
    for doc in tqdm.tqdm(docs):
        doc = strip_html_tags(doc)
        doc = doc.translate(doc.maketrans("\n\t\r", "   "))
        doc = doc.lower()
        doc = remove_accented_chars(doc)
        doc = contractions.fix(doc)
        # lower case and remove special characters\whitespaces
        doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
        doc = re.sub(' +', ' ', doc)
        doc = doc.strip()  
        norm_docs.append(doc)
  
    return norm_docs


In [9]:
import tqdm
import re
from bs4 import BeautifulSoup
import unicodedata
import contractions
X = pre_process_corpus(X)


  0%|          | 0/3600000 [00:00<?, ?it/s]/tmp/ipykernel_4224/2395363686.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")
  5%|▍         | 174543/3600000 [00:16<05:11, 10980.14it/s]/tmp/ipykernel_4224/2395363686.py:2: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")
100%|██████████| 3600000/3600000 [05:37<00:00, 10680.16it/s]


In [43]:
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 1.8 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 4.7 MB/s eta 0:00:00


In [10]:
#Normalizacja - były klasy 1,2 teraz sa 0 i 1
y=y-1



In [11]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# Tokenizacja tekstu
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
word_index = tokenizer.word_index
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(sequences, y, test_size=0.3, random_state=42)

max_sequence_length = max(len(seq) for seq in X)
X_train = pad_sequences(X_train, maxlen=max_sequence_length)
X_val = pad_sequences(X_val, maxlen=max_sequence_length)

Twoim zadaniem jest stworzenie modelu sieci neuronowej, który będzie klasyfikować recenzje na podstawie tytułu. W danych mamy dostarczone klasy (1) albo (2). 
Kroki, które musisz podjąć:

1. Przygotowanie danych: Dane zostały wczytane z pliku CSV. Teksty znajdują się w kolumnach title, a etykiety w kolumnie "class". Teksty oraz etykiety są przygotowane do dalszej obróbki.

2. Tokenizacja tekstu: Teksty powinny być tokenizowane za pomocą Tokenizer, a następnie przekształcane na sekwencje liczb. Jest to niezbędne, aby można było używać ich jako danych wejściowych do modelu sieci neuronowej.

3. Zbuduj model sieci neuronowej: Model sieci rekurencyjnej zdefiniuj w kodzie i przykładowo niech składa się z następujących warstw:

* Warstwa wejściowa: Przyjmuje sekwencje tokenów o długości określonej przez max_sequence_length.
* Warstwa Embedding: Mapuje tokeny na wektory o wymiarze embed_dim (100 w tym przypadku), co pozwala na reprezentację słów jako gęstych wektorów cech.
* Warstwa MultiHeadAttention: Implementuje mechanizm uwagi, który pozwala modelowi skupić się na różnych częściach sekwencji wejściowej, co jest szczególnie przydatne w przetwarzaniu języka naturalnego.
* GlobalAveragePooling1D: Agreguje informacje z różnych części sekwencji, redukując wymiarowość danych i przygotowując je do procesu klasyfikacji.
* Warstwa PositionalEncoding(position, embed_dim)(embedding_layer), która mapuje sekwencje na odpowiednie pozycje. 
* Warstwa Dense (wyjściowa): Z funkcją aktywacji softmax, generuje dystrybucję prawdopodobieństwa dla dwóch możliwych klas sentymentu, umożliwiając klasyfikację.


4. Kompilacja modelu: Model jest kompilowany z optymalizatorem Adam, funkcją straty "categorical_crossentropy" (odpowiednia do klasyfikacji wieloklasowej) oraz metryką "accuracy".

5. Trenowanie modelu: Model jest trenowany na danych tweetów przez 5 epok, z wsadem o rozmiarze 32. Wynik trenowania jest oceniany na zbiorze walidacyjnym. 

6. Porównaj swój model z https://www.kaggle.com/code/yacharki/binary-classification-amazon-reviews-84-lstm/notebook#Text-classification-with-an-RNN-%5BLSTM%5D. 

Twoim zadaniem jest uzupełnienie kodu i dostosowanie parametrów modelu oraz trenowania, aby uzyskać jak najlepsze wyniki w zadaniu klasyfikacji. Możesz eksperymentować z parametrami, takimi jak liczba epok trenowania, rozmiar wsadu (batch size).


In [14]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Embedding, MultiHeadAttention, GlobalAveragePooling1D, Dense
import numpy as np
embed_dim = 100  # Wymiar wektora embeddingowego
num_heads = 2  # Liczba "głowic" w warstwie Multi-Head Attention
vocab_size = 10000  # Rozmiar słownika

inputs = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(input_dim=vocab_size, output_dim=embed_dim)(inputs)
attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(embedding_layer, embedding_layer)
global_average = GlobalAveragePooling1D()(attention_output)
#global_average= Dropout(0.5) (global_average)
outputs = Dense(1, activation='sigmoid')(global_average)

model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=600, validation_split=0.2, verbose=1, shuffle=True)

Epoch 1/10
3360/3360 [==============================] - 222s 66ms/step - loss: 0.3748 - accuracy: 0.8244 - val_loss: 0.3376 - val_accuracy: 0.8430
Epoch 2/10
3360/3360 [==============================] - 206s 61ms/step - loss: 0.3319 - accuracy: 0.8446 - val_loss: 0.3289 - val_accuracy: 0.8463
Epoch 3/10
3360/3360 [==============================] - 205s 61ms/step - loss: 0.3241 - accuracy: 0.8482 - val_loss: 0.3256 - val_accuracy: 0.8488
Epoch 4/10
3360/3360 [==============================] - 205s 61ms/step - loss: 0.3192 - accuracy: 0.8507 - val_loss: 0.3240 - val_accuracy: 0.8493
Epoch 5/10
3360/3360 [==============================] - 205s 61ms/step - loss: 0.3156 - accuracy: 0.8528 - val_loss: 0.3219 - val_accuracy: 0.8503
Epoch 6/10
3360/3360 [==============================] - 205s 61ms/step - loss: 0.3126 - accuracy: 0.8542 - val_loss: 0.3234 - val_accuracy: 0.8510
Epoch 7/10
3360/3360 [==============================] - 204s 61ms/step - loss: 0.3096 - accuracy: 0.8558 - val_loss: 0